In [317]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Rohdaten importieren
excel_file_path = '2023_eim.xlsx'
all_sheets_dict = pd.read_excel(excel_file_path, sheet_name=None)

# leere Liste erstellen für die DataFrames
all_sheets_list = []

# Schleife über alle DataFrames in all_sheets_dict und diese in die Liste all_sheets_list einfügen
for sheet_name, sheet_df in all_sheets_dict.items():
    # Spalte 'Jahr' aus dem Sheet-Namen erstellen
    sheet_df['Jahr'] = sheet_name
    all_sheets_list.append(sheet_df)

# Zusammenführen der DataFrames in all_sheets_list zu einem DataFrame
combined_df = pd.concat(all_sheets_list, ignore_index=True)

# Speichern des DataFrames in eine Excel-Datei
combined_df.to_excel('2023_eim_combined.xlsx', index=False)

# Erstellen eines neuen DataFrames mit der Anzahl der einzigartigen Kunden und der Summe der Tage, gruppiert nach 'Mitarbeiter' und 'Jahr'
grouped_df = combined_df.groupby(['Mitarbeiter', 'Jahr']).agg(
    num_customers=pd.NamedAgg(column='Anforderer', aggfunc='nunique'),
    sum_tage=pd.NamedAgg(column='Tage', aggfunc='sum')
).reset_index()

# Speichern des gruppierten DataFrames in eine Excel-Datei
grouped_df.to_excel('2023_eim_grouped.xlsx', index=False)

Die gruppierten Daten dienen als Input für die weitere Aufbereitung der Daten, zu einer Excel Datei mit den Input Features.

In [318]:
# import input features
input_file_path = 'input_features.xlsx'

input_df = pd.read_excel(input_file_path)
arbeitstage = {2017: 247, 2018: 249, 2019: 249, 2020: 253, 2021: 253, 2022: 250, 2023: 248}
bip_deutschland = {2017: 2.7, 2018: 1, 2019: 1.1, 2020: -3.8, 2021: 3.2, 2022: 1.8, 2023: -0.3}
fis_turnover_per_employee = {2017: 98658, 2018: 102865, 2019: 104651, 2020: 107317, 2021: 109112, 2022: 111318, 2023: 114443}

# add column 'Arbeitstage' to the dataframe according to the column 'Jahr'
input_df['Arbeitstage'] = input_df['Jahr'].map(arbeitstage)
# add column 'BIP Deutschland' to the dataframe according to the column 'Jahr'
input_df['Wirtschaftswachstum'] = input_df['Jahr'].map(bip_deutschland)
# add column 'FIS Turnover per Employee' to the dataframe according to the column 'Jahr'
input_df['FIS Umsatz pro Mitarbeiter'] = input_df['Jahr'].map(fis_turnover_per_employee)
# multiply the column 'Vollzeit' with the column 'Arbeitstage' and add the result to the dataframe as column 'Workdays'
input_df['Workdays'] = input_df['Vollzeit'] * input_df['Arbeitstage']

# print the head of the dataframe
print(input_df.head())

   Jahr   Name  Alter  Berufserfahrung  Anzahl Kunden  Bestandskunden  Level  \
0  2017  BAERJ    NaN              NaN             45             NaN    NaN   
1  2018  BAERJ    NaN              NaN             48             NaN    NaN   
2  2019  BAERJ    NaN              NaN             54             NaN    NaN   
3  2020  BAERJ    NaN              NaN             50             NaN    NaN   
4  2021  BAERJ    NaN              NaN             36             NaN    NaN   

   Fehltage  Vollzeit    Tage  Arbeitstage  Wirtschaftswachstum  \
0       NaN       NaN  112.17          247                  2.7   
1       NaN       NaN  123.63          249                  1.0   
2       NaN       NaN   74.69          249                  1.1   
3       NaN       NaN  130.47          253                 -3.8   
4       NaN       NaN  132.96          253                  3.2   

   FIS Umsatz pro Mitarbeiter  Workdays  
0                       98658       NaN  
1                      102865   

Kalkuliere welche Kunden die Berater in jeweiligen Jahr betreut haben.
Es soll nach Höhe des Aufwands geclustert werden.
Hat ein Berater einen Kunden mit mehr als 80 Tagen, so soll dieser Kunde in die Kategorie A fallen.
Hat ein Berater einen Kunden mit mehr als 40 Tagen, so soll dieser Kunde in die Kategorie B fallen.
Hat ein Berater einen Kunden mit weniger als 40 Tagen, so soll dieser Kunde in die Kategorie C fallen.
Hat ein Berater nur Kunden mit weniger als 10 Tagen soll diser in die Kategorie D fallen.

In [319]:
# based on the combined_df dataframe, create a new dataframe with the customers and the categories A, B, C, D per Name

# first filter the combined_df dataframe, select the highest value of 'Tage' for each 'Mitarbeiter' and 'Jahr'
customers_filtered_highest_df = combined_df.loc[combined_df.groupby(['Mitarbeiter', 'Jahr'])['Tage'].idxmax()]
# add a column 'Kategorie' to the dataframe
customers_filtered_highest_df['Kategorie'] = ''
# set 'Kategorie' to 'D' if 'Tage' is less than 10 for the row in customers_filtered_highest_df
customers_filtered_highest_df.loc[customers_filtered_highest_df['Tage'] <= 10, 'Kategorie'] = 'D'
# set 'Kategorie' to 'C' if 'Tage' is greater than 40 for the row in customers_filtered_highest_df
customers_filtered_highest_df.loc[customers_filtered_highest_df['Tage'] > 10, 'Kategorie'] = 'C'
# set 'Kategorie' to 'B' if 'Tage' is greater than 40 for the row in customers_filtered_highest_df
customers_filtered_highest_df.loc[customers_filtered_highest_df['Tage'] > 40, 'Kategorie'] = 'B'
# set 'Kategorie' to 'A' if 'Tage' is greater than 80 for the row in customers_filtered_highest_df
customers_filtered_highest_df.loc[customers_filtered_highest_df['Tage'] > 80, 'Kategorie'] = 'A'
# set 'Kategorie' to 'B' if 'Tage' is greater than 40 for the row in customers_filtered_highest_df

# save customers_filtered_highest_df to an excel file
customers_filtered_highest_df.to_excel('customers_filtered_highest.xlsx', index=False)


In [322]:
# map the 'Kategorie' of customers_filtered_highest_df to the input_df dataframe based on 'Mitarbeiter' and 'Jahr' where 'Mitarbeiter' equals 'Name' in the input_df dataframe
input_df['Kategorie'] = input_df.apply(lambda row: customers_filtered_highest_df.loc[(customers_filtered_highest_df['Mitarbeiter'] == row['Name']) & (customers_filtered_highest_df['Jahr'] == row['Jahr']), 'Kategorie'].iloc[0], axis=1)  



IndexError: single positional indexer is out-of-bounds

In [0]:
# filter the data in the dataframe input_df only to show data where alter is not NaN
input_df = input_df[input_df['Alter'].notna()]
print(input_df)

In [0]:
# save the dataframe to an excel file
input_df.to_excel('input_features_filtered.xlsx', index=False)

Aufbau eines Models mit den Input Features.
Jahr, Alter, Berufserfahrung, Anzahl Kunden, Bestandskunden
Die Target Variable ist die Variable Tage.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Prepare the data for training the model
X = input_df[['Alter', 'Berufserfahrung', 'Anzahl Kunden', 'Bestandskunden', 'FIS Umsatz pro Mitarbeiter', 'Workdays', 'Level']]
y = input_df['Tage']

# Split the data into training and cross validation data - 80% training, 20% cross validation
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# save the model to disk
import pickle
filename = 'bt_model.sav'
pickle.dump(model, open(filename, 'wb'))

# Evaluate the model with the cross validation data
y_pred_cv = model.predict(X_cv)
# Evaluate the model with the training data
y_pred_train = model.predict(X_train)

# Mean Squared Error with the cross validation data
mse_cv = mean_squared_error(y_cv, y_pred_cv)
# Mean Squared Error with the training data
mse_train = mean_squared_error(y_train, y_pred_train)
# Calculate the Mean Absolute Error with the cross validation data
mae_cv = np.mean(np.abs(y_cv - y_pred_cv))
# Calculate the Mean Absolute Error with the training data
mae_train = np.mean(np.abs(y_train - y_pred_train))
# Root Mean Squared Error with the cross validation data
rmse_cv = np.sqrt(mse_cv)
# Root Mean Squared Error with the training data
rmse_train = np.sqrt(mse_train)
print(f'Mean Squared Error CV: {mse_cv}')
print(f'Mean Absolute Error CV: {mae_cv}')
print(f'Root Mean Squared Error CV: {rmse_cv}')
print(f'Mean Squared Error Train: {mse_train}')
print(f'Mean Absolute Error Train: {mae_train}')
print(f'Root Mean Squared Error Train: {rmse_train}')

In [ ]:
# print the y_cv and y_pred_cv values and compare them in a dataframe
output_df = pd.DataFrame({'y_cv': y_cv, 'y_pred_cv': y_pred_cv})
print(output_df)

Zeige die Korellation der Input Features mit der Target Variable in je einem Scatter Plot.  

In [ ]:
# show the correlation of the input features with the target variable in a scatter plot
fig, axs = plt.subplots(3, 3, figsize=(20, 20))
axs[0, 0].scatter(X['Alter'], y)
axs[0, 0].set_title('Alter')
axs[0, 0].set_xlabel('Alter')
axs[0, 0].set_ylabel('Beratertage')
axs[0, 1].scatter(X['Berufserfahrung'], y)
axs[0, 1].set_title('Berufserfahrung')
axs[0, 2].scatter(X['Workdays'], y)
axs[0, 2].set_title('Workdays')
axs[1, 0].scatter(X['Anzahl Kunden'], y)
axs[1, 0].set_title('Anzahl Kunden')
axs[1, 1].scatter(X['Bestandskunden'], y)
axs[1, 1].set_title('Bestandskunden')
#axs[1, 2].scatter(X['Wirtschaftswachstum'], y)
#axs[1, 2].set_title('Wirtschaftswachstum')
axs[2, 0].scatter(X['FIS Umsatz pro Mitarbeiter'], y)
axs[2, 0].set_title('FIS Umsatz pro Mitarbeiter')
axs[2, 1].scatter(X['Level'], y)
axs[2, 1].set_title('Level')

plt.show()

In [ ]:
import seaborn as sns

correlation_matrix = X_train.corr()

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Draw the heatmap
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", linewidths=.5)

# Add title
plt.title('Correlation Matrix')

# Show the plot
plt.show()

In [ ]:
# Calculate correlation of each independent variable in X with the target variable y
correlation = X.corrwith(y)

# create a bar plot of the correlation of each independent variable with the target variable
plt.figure(figsize=(10, 6))
correlation.plot(kind='bar', color='blue')
# plot the values of each bar on the bar plot
for index, value in enumerate(correlation):
    plt.text(index, value, str(round(value, 2)))
plt.title('Correlation of the Features with the Target Variable')
plt.xlabel('Features')
plt.ylabel('Correlation Coefficient')
plt.show()

In [ ]:
# create a pearson correlation test
from scipy.stats import pearsonr

# calculate the pearson correlation coefficient and the p-value for the correlation for every feature within X
for column in X:
    corr, p = pearsonr(X[column], y)
    print(f'Pearsons correlation coefficient between {column} and y: {corr}')
    print(f'p-value: {p}')
    if p > 0.05:
        print('The correlation is not significant')
    else:
        print('The correlation is significant')
    print('.......................................................')

Nachdem das Model mit Hilfe von Linearer Regression erstellt wurde, wird nun ein Model mit Hilfe eines neuronalen Netzes erstellt.

In [ ]:
import tensorflow as tf
#from sklearn.preprocessing import StandardScaler
import keras
from keras import layers

#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

# Split the data into training and cross validation data - 80% training, 20% cross validation
X_train_nn, X_cv_nn, y_train_nn, y_cv_nn = train_test_split(X, y, test_size=0.2, random_state=42)

print(
    f"Using {len(X_train_nn)} records for training "
    f"and {len(X_cv_nn)} records for cross validation")

In [ ]:
# build the neural network model
nn_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1)
])

# compile the model
nn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# train the model
nn_model.fit(X_train_nn, y_train_nn, epochs=2000)


In [ ]:
# evaluate the model with the cross validation data
print("Results for the cross validation data:")
print(nn_model.evaluate(X_cv_nn, y_cv_nn))

# evaluate the model with the training data
print("Results for the training data:")
print(nn_model.evaluate(X_train_nn, y_train_nn))

print(".......................................................")
print("Results from the linear regression model:")
print(f'Mean Squared Error CV: {mse_cv}')
print(f'Mean Absolute Error CV: {mae_cv}')
print(f'Root Mean Squared Error CV: {rmse_cv}')
print(f'Mean Squared Error Train: {mse_train}')
print(f'Mean Absolute Error Train: {mae_train}')
print(f'Root Mean Squared Error Train: {rmse_train}')

In [ ]:
# save the tensorflow model to disk
nn_model.save('tf_bt_model.keras')